In [1]:
#Importing the neccessary modules
import tensorflow as tf
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#the training and testing datasets in csv format
train_data = "fashion-mnist_train.csv"
test_data = "fashion-mnist_test.csv"

In [3]:
#reading the train_data and test_data usign the pandas libary
train = pd.read_csv(train_data)
test = pd.read_csv(test_data)

In [5]:
#print("Test\n\n", train.head() , "\n\n")
#print("Train\n\n" , test.head() , "\n\n")

In [6]:
#function to get the features and labels from the train and test data
def get_features_labels(data):
    features = data.values[:, 1:]/255
    labels = data['label'].values
    return features, labels

In [7]:
#getting the features and labels 
train_features, train_labels = get_features_labels(train)
test_features, test_labels = get_features_labels(test)

#printing their shapes
print(train_features.shape)
print(test_features.shape)


(60000, 784)
(10000, 784)


In [8]:
# Building the keras model with the relu and softmax activation fuctions
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu, input_shape=(784,)))
model.add(tf.keras.layers.Dense(20, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

#Compiling the model and printing the summary()
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.summary()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                23550     
_________________________________________________________________
dense_1 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
Total params: 24,380
Trainable params: 24,380
Non-trainable params: 0
_________________________________________________________________


In [9]:
#encoding the labels becuase i am using categorical_crossentropy in compiling the model
train_labels_1hot = tf.keras.utils.to_categorical(train_labels)
test_labels_1hot = tf.keras.utils.to_categorical(test_labels)

model.fit(train_features, train_labels_1hot, epochs=5, batch_size=128)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 19s 319us/sample - loss: 0.6726 - acc: 0.7700
Epoch 2/5
60000/60000 [==============================] - 13s 220us/sample - loss: 0.4594 - acc: 0.8378
Epoch 3/5
60000/60000 [==============================] - 13s 223us/sample - loss: 0.4193 - acc: 0.8525
Epoch 4/5
60000/60000 [==============================] - 13s 222us/sample - loss: 0.3960 - acc: 0.8582
Epoch 5/5
60000/60000 [==============================] - 32s 537us/sample - loss: 0.3785 - acc: 0.8664


In [10]:
#evaluate the model and print its loss and accuracy
test_loss, test_accuracy = model.evaluate(test_features, test_labels_1hot)

print("Test Loss = ", test_loss)
print("Test accuracy = ", test_accuracy)

10000/10000 [==============================] - 2s 194us/sample - loss: 0.3774 - acc: 0.8661
Test Loss =  0.3774197867155075
Test accuracy =  0.8661


In [11]:
#Then i save the model trained with keras
model.save('keras_model.h5')

In [12]:
#converting the keras model to estimator model
tf_classifier_model = tf.keras.estimator.model_to_estimator(keras_model=model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Javis\\AppData\\Local\\Temp\\tmpwfxeq1rk', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster

In [13]:
input_name = model.input_names[0]

#preparing input for training the model with tf.estimator
train_input_fn = tf.estimator.inputs.numpy_input_fn(x={input_name: train_features}, y=train_labels_1hot, batch_size=128, 
                                                    num_epochs=5, shuffle=True)
#getting input names
feature_columns = [tf.feature_column.numeric_column(input_name, shape=784)]

In [14]:
classifier = tf_classifier_model

In [15]:
#training the model
classifier.train(input_fn=train_input_fn)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Warm-starting with WarmStartSettings: WarmStartSettings(ckpt_to_initialize_from='C:\\Users\\Javis\\AppData\\Local\\Temp\\tmpwfxeq1rk\\keras\\keras_model.ckpt', vars_to_warm_start='.*', var_name_to_vocab_info={}, var_name_to_prev_var_name={})
INFO:tensorflow:Warm-starting from: C:\Users\Javis\AppData\Local\Temp\tmpwfxeq1rk\keras\keras_model.ckpt
INFO:tensorflow:Warm-starting variables only in TRAINABLE_VARIABLES.
INFO:tensorflow:Warm-started 6 variables.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.wher

In [16]:
#Evaluating the model

evaluate_input = tf.estimator.inputs.numpy_input_fn(x={input_name: test_features}, y=test_labels_1hot, num_epochs=1, 
                                                    shuffle=False)

classifier.evaluate(input_fn=evaluate_input)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-12-20T23:24:02Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Javis\AppData\Local\Temp\tmpwfxeq1rk\model.ckpt-2344
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-12-20-23:24:04
INFO:tensorflow:Saving dict for global step 2344: acc = 0.876, global_step = 2344, loss = 0.34729847
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2344: C:\Users\Javis\AppData\Local\Temp\tmpwfxeq1rk\model.ckpt-2344


{'acc': 0.876, 'loss': 0.34729847, 'global_step': 2344}

In [17]:
#save the model trianed with the estimator
feature_spec = {
    input_name: tf.FixedLenFeature(shape=[784], dtype=np.float32)
}
serving_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
export_dir = classifier.export_savedmodel(export_dir_base="", 
                            serving_input_receiver_fn=serving_fn)
export_dir = export_dir.decode("utf8")

Instructions for updating:
This function has been renamed, use `export_saved_model` instead.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from C:\Users\Javis\AppData\Local\Temp\tmpwfxeq1rk\model.ckpt-2344
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: temp-b'1576880649'\saved_model.pb
